from [here](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
# # cd /content/gdrive/MyDrive/English_Hindi/all_files
# # #ls
# # #!unzip 'Data_text.zip'
# # %cd '/content/gdrive/MyDrive/English_Hindi/all_files/Data_text'

# # #!pip install googletrans==4.0.0rc1

# #import pandas as pd
# #data1=pd.read_csv('tanglish_kaggle.csv',encoding='utf-8',error_bad_lines=False,sep='\t')

# f = open("x1_tamil.txt",'r',encoding = 'utf-8')
# apple=f.read()
# apple=apple.split('\n')

# from googletrans import Translator
# translator = Translator()

# list1=iter(range(0,1000))

# with open("x1_english_new.txt",'w') as f:
#   for i in apple:
#     print(next(list1))
#     if i=="":
#       f.write("")
#       f.write('\n')
#       continue
#     try:
#       output = translator.translate(i, src='ta')
#       f.write(output.text)
#       f.write('\n')
#     except:
#       f.write("")
#       f.write('\n')
#   f.close


In [35]:
f = open("x2_english.txt",'r')
apple=f.read()
apple=apple.split('\n')

In [37]:
f = open("x2_tanglish.txt",'r')
ball=f.read()
ball=ball.split('\n')

In [38]:
len(apple)

183451

In [39]:
len(ball)

183452

In [51]:
apple[1]

'similar conditions will be imposed if the sri lankan government is given an imf loan .'

In [45]:
#ball.pop()

In [44]:
len(ball)

183451

In [46]:
import pandas as pd

In [47]:
new_data=pd.DataFrame()

In [ ]:
new_data['tanglish']=

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
cd /content/gdrive/MyDrive/English_Hindi/Translation

/content/gdrive/.shortcut-targets-by-id/1x3R88qmTn1B0aC9FOyKKsxS-yecowjy7/Translation


In [ ]:
ls

2000_pred_translations5.xlsx  second_phase-Improved.ipynb
final_100_row_data.xlsx       tamil.txt
manual_tanglish_english.csv   tvs_tamil_Total100_06122020.xlsx


### 1. Data Reading

In [ ]:
# data = pd.read_excel('non_lotus_100.xlsx')

In [ ]:
data = pd.read_excel('final_100_row_data.xlsx')

In [ ]:
data = data.drop(columns = ['Unnamed: 0'])

In [ ]:
data

,English_data,Tamil_data
0,At Lotus good response was given,Lotus la nalla responce kuduthainga
1,Selvakumar attended,selvakumar tha atten painnaru
2,Showroom is good,showroom nalla iruku
3,"Saw the vehicle, looks good",vandi pathom nalla iruku
4,Waiting for MD's approval,MD Approval ku wait painnarom
...,...,...
396,They explained the details,details ellam explain painnaga
397,Lotus showroom is good,lotus showroom nalla iruku
398,Sales persons gave good response,sales person nalla response kuduthainga
399,Vehicle is good,vandi nalla iruku


In [ ]:
import re
for ind,text in enumerate(data['English_data']):
    data['English_data'][ind] = re.sub(r'[^A-Za-z]+', ' ', text.lower())
    
import re
for ind,text in enumerate(data['Tamil_data']):
    data['Tamil_data'][ind] = re.sub(r'[^A-Za-z]+', ' ', text.lower())

In [ ]:
# import pandas as pd
# data1 = pd.read_csv('/content/gdrive/MyDrive/English_Hindi/Translation/manual_tanglish_english.csv')


In [ ]:
# str1=""
# for i in data1['Tanglish']:
#   str1=str1+i

In [ ]:
# text_file = open("/content/gdrive/MyDrive/English_Hindi/Bigger_Corpus.txt", "w")
# n = text_file.write(str1)
# text_file.close()

In [ ]:
import re, collections

def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1

    return vocab

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token]

    string_tokens = []
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        token_reg = re.escape(token.replace('.', '[.]'))

        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        if len(matched_positions) == 0:
            continue
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    return string_tokens

# vocab = {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3}

vocab = get_vocab('tamil.txt')

print('==========')
print('Tokens Before BPE')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
print('All tokens: {}'.format(tokens_frequencies.keys()))
print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
print('==========')

num_merges = 600
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)


Tokens Before BPE
All tokens: dict_keys(['l', 'o', 't', 'u', 's', '</w>', 'a', 'n', 'r', 'e', 'p', 'c', 'k', 'd', 'h', 'i', 'g', 'v', 'm', 'w', 'x', 'b', 'f', 'y', 'z', 'j', 'q'])
Number of tokens: 27


In [ ]:
# Let's check how tokenization will be for a known word
word_given_known = 'nalla</w>'
# word_given_unknown = 'vilakkamnallasonnaga</w>'
word_given_unknown = 'cheppu</w>'


sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

print(sorted_tokens)

word_given = word_given_known 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

word_given = word_given_unknown 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

['yedukaramathiritha</w>', 'kuduthaganutha</w>', 'specification</w>', 'kuduthahinaga</w>', 'painnarainga</w>', 'construction</w>', 'kudhuthainga</w>', 'kudukarainga</w>', 'vechuruainga</w>', 'iruindhainga</w>', 'kuduthainga</w>', 'irundhainga</w>', 'peasanainga</w>', 'kuduthainag</w>', 'soileerukom</w>', 'vandhapodhu</w>', 'irunadhinga</w>', 'soinnainga</w>', 'painnainga</w>', 'irundhudhu</w>', 'kudthainga</w>', 'kudutainga</w>', 'selvakumar</w>', 'mudinjutha</w>', 'kuduthaiga</w>', 'kuduthanga</w>', 'anusarichu</w>', 'kudukarenu</w>', 'kuudthanga</w>', 'soinnainag</w>', 'katanainga</w>', 'kuudukalam</w>', 'irundainga</w>', 'irukainga</w>', 'kudukalam</w>', 'pathainga</w>', 'kudutharu</w>', 'quotation</w>', 'satisfied</w>', 'irundharu</w>', 'vandhuten</w>', 'painnarom</w>', 'satisfide</w>', 'kudutahru</w>', 'paunainga</w>', 'soinainga</w>', 'soinnaiga</w>', 'documents</w>', 'vishiyama</w>', 'varathula</w>', 'nadakudhu</w>', 'staisfide</w>', 'vainguvum</w>', 'response</w>', 'showroom</w

### 2. Data Preparation

#### Encoding each word to a  number. we will trim the vector at the end to limit the computation.

In [ ]:
SOS_token = 0
EOS_token = 1
UNK_token = 2 #change


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"SOS":0,"EOS":1,"UNK":2} #change
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"UNK"}#change
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence): #function change
        new_sentence = ''
        for word in sentence.split(' '):
            unk_word = self.addWord(word)
            if not new_sentence:
                new_sentence =unk_word
            else:
                new_sentence = new_sentence + ' ' + unk_word
        return new_sentence
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

#### save file name in  'data/%s-%s.txt' % (lang1, lang2) format.

In [ ]:
def readLangs(lang1, lang2, reverse=True): #from some language to English
    print("Reading lines...")
    
    
    pairs = [[i,j] for i,j in zip(data['English_data'],data['Tamil_data'])]
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2) #tamil
        output_lang = Lang(lang1) #english
    else:
        input_lang = Lang(lang1) #english
        output_lang = Lang(lang2) #tamil

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 40


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        #print(pair[0])
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        #break
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('English','Tamil', True)
print(random.choice(pairs))

Reading lines...
Read 401 sentence pairs
Trimmed to 401 sentence pairs
Counting words...
Counted words:
Tamil 286
English 237
['sales person nalla response kuduthanga', 'sales person gave good response']


### 3. Model Building

In [ ]:
### Encoder

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
### Decoder

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [ ]:
### Attention Decoder

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### 4. Training

In [ ]:
#Preparing Training Data

def wordindex(lang,word):
    try:
        return lang.word2index[word]
    except:
        return lang.word2index["UNK"]
    
def indexesFromSentence(lang, sentence):
    return [wordindex(lang,word) for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

### 5. Evaluation

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=20):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 3000, print_every=500)


0m 12s (- 1m 3s) (500 16%) 2.7122
0m 25s (- 0m 50s) (1000 33%) 1.9971
0m 38s (- 0m 38s) (1500 50%) 1.7772
0m 51s (- 0m 25s) (2000 66%) 1.5607
1m 4s (- 0m 12s) (2500 83%) 1.2701
1m 16s (- 0m 0s) (3000 100%) 1.0082


In [ ]:
evaluateRandomly(encoder1, attn_decoder1) # 3000

> vandi details ellam clear ah explain painnainga
= they gave a clear explanation on vehicle details
< they gave a very clear explanation on vehicle details <EOS>

> showroom nalla iruku
= showroom is good
< showroom is good <EOS>

> vandi nalla iruku
= vehicle is good
< vehicle is good <EOS>

> vandi details ellam romba claer ah explain painnainga
= gave a very clear explanation on the vehicle details
< they gave a very clear explanation on vehicle details <EOS>

> lunch time la vandi pakka ponom
= we went to see the vehicle during lunch time
< we went to the vehicle <EOS>

> nalla response kudutainga
= they gave good response
< they gave good response <EOS>

> showroom nalla iruku
= showroom is good
< showroom is good <EOS>

> showroom nalla iruku
= showroom is good
< showroom is good <EOS>

> nalla response kudutharu
= he gave good response
< selva gave good response <EOS>

> nalla friendly ya irundharu
= he was friendly
< he was friendly <EOS>

> lotus la nalla response kuduthainga

In [ ]:
row = "quotation matu tha vangirukom"
output_words, attentions = evaluate(encoder1, attn_decoder1, row)
output_sentence = ' '.join(output_words)
print(row,'=', output_sentence)

quotation matu tha vangirukom = lotus lotus only attended <EOS>


In [ ]:
row = "vandi yadukara idea la ila explanation la nala tha kuduthanga"
output_words, attentions = evaluate(encoder1, attn_decoder1, row)
output_sentence = ' '.join(output_words)
print(row,'=', output_sentence)

vandi yadukara idea la ila explanation la nala tha kuduthanga = lotus only attended good response at lotus <EOS>


## BPE

In [ ]:
import re, collections

def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1

    return vocab

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token]

    string_tokens = []
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        token_reg = re.escape(token.replace('.', '[.]'))

        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        if len(matched_positions) == 0:
            continue
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    return string_tokens

# vocab = {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3}

vocab = get_vocab('tamil.txt')

print('==========')
print('Tokens Before BPE')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
print('All tokens: {}'.format(tokens_frequencies.keys()))
print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
print('==========')

num_merges = 600
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
    print('All tokens: {}'.format(tokens_frequencies.keys()))
    print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
    print('==========')

Tokens Before BPE
All tokens: dict_keys(['l', 'o', 't', 'u', 's', '</w>', 'a', 'n', 'r', 'e', 'p', 'c', 'k', 'd', 'h', 'i', 'g', 'v', 'm', 'w', 'x', 'b', 'f', 'y', 'z', 'j', 'q'])
Number of tokens: 27
Iter: 0
Best pair: ('a', '</w>')
All tokens: dict_keys(['l', 'o', 't', 'u', 's', '</w>', 'a</w>', 'n', 'a', 'r', 'e', 'p', 'c', 'k', 'd', 'h', 'i', 'g', 'v', 'm', 'w', 'x', 'b', 'f', 'y', 'z', 'j', 'q'])
Number of tokens: 28
Iter: 1
Best pair: ('l', 'a</w>')
All tokens: dict_keys(['l', 'o', 't', 'u', 's', '</w>', 'la</w>', 'n', 'a', 'r', 'e', 'p', 'c', 'k', 'd', 'h', 'i', 'g', 'a</w>', 'v', 'm', 'w', 'x', 'b', 'f', 'y', 'z', 'j', 'q'])
Number of tokens: 29
Iter: 2
Best pair: ('n', 'a')
All tokens: dict_keys(['l', 'o', 't', 'u', 's', '</w>', 'la</w>', 'na', 'r', 'e', 'p', 'n', 'c', 'k', 'd', 'h', 'a', 'i', 'g', 'a</w>', 'v', 'm', 'w', 'x', 'b', 'f', 'y', 'z', 'j', 'q'])
Number of tokens: 30
Iter: 3
Best pair: ('l', 'la</w>')
All tokens: dict_keys(['l', 'o', 't', 'u', 's', '</w>', 'la</w>',

In [ ]:
# Let's check how tokenization will be for a known word
word_given_known = 'nalla</w>'
# word_given_unknown = 'vilakkamnallasonnaga</w>'
word_given_unknown = 'lanallaresponcekuduthainga</w>'


sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

print(sorted_tokens)

word_given = word_given_known 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

word_given = word_given_unknown 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

['yedukaramathiritha</w>', 'kuduthaganutha</w>', 'specification</w>', 'kuduthahinaga</w>', 'painnarainga</w>', 'construction</w>', 'kudhuthainga</w>', 'kudukarainga</w>', 'vechuruainga</w>', 'iruindhainga</w>', 'kuduthainga</w>', 'irundhainga</w>', 'peasanainga</w>', 'kuduthainag</w>', 'soileerukom</w>', 'vandhapodhu</w>', 'irunadhinga</w>', 'soinnainga</w>', 'painnainga</w>', 'irundhudhu</w>', 'kudthainga</w>', 'kudutainga</w>', 'selvakumar</w>', 'mudinjutha</w>', 'kuduthaiga</w>', 'kuduthanga</w>', 'anusarichu</w>', 'kudukarenu</w>', 'kuudthanga</w>', 'soinnainag</w>', 'katanainga</w>', 'kuudukalam</w>', 'irundainga</w>', 'irukainga</w>', 'kudukalam</w>', 'pathainga</w>', 'kudutharu</w>', 'quotation</w>', 'satisfied</w>', 'irundharu</w>', 'vandhuten</w>', 'painnarom</w>', 'satisfide</w>', 'kudutahru</w>', 'paunainga</w>', 'soinainga</w>', 'soinnaiga</w>', 'documents</w>', 'vishiyama</w>', 'varathula</w>', 'nadakudhu</w>', 'staisfide</w>', 'vainguvum</w>', 'response</w>', 'showroom</w

### For a sample to next 20 rows

In [ ]:
new = ["quotation matu tha vangirukom  vandi yadukara idea la ila explanation la nala tha kuduthanga",
"explanation la nala va sonaru katayellama sonaru week end la varen thank you",
"explanation la nala tha kuduthanga details songa  thank you",
"explanation la nala kudutharu sales man details sonaga free time la vara vanthu vandi yadukaren nane call pandren varu pothu thank you",
"thavayana yellama sonaga details yellama kuduthanga nxt week kula vandi ya yadukaren thank you",
"salesman nallatha explain pannanga eppo vandi eadukara mari illa eadukum pothu angave vanthu eaduthukaran",
"sales person Neata attend pannaga vandi Vilakkam nalla Sonnaga Entha Problem Illa Testride Koduthaga Amount Ready pannittu 2 or 3 days la Varen",
"Showroom La sales person Neata Welcome panni vandi details nalla Sonnaga Nalla respones pannaga  No Problem Diwali Ku varen",
"Showroom La sales person Neata attend pannaga Vandi Vilakkam Nalla Sonnaga vandi Testride Koduthanga No Problem",
"Showroom La Ellamea neata panni Koduthaga Vandi details Nalla sonnaga Amount Reday pannittu Next week la Varen",
"sales person Neata attend pannaga vandi details sonnaga testride koduthaga  Ellamea neata Panni Koduthaga friday vandi eduthu Karen"]

In [ ]:
for row in new:
    output_words, attentions = evaluate(encoder1, attn_decoder1, row)
    output_sentence = ' '.join(output_words)
    print(row,'=', output_sentence)

quotation matu tha vangirukom  vandi yadukara idea la ila explanation la nala tha kuduthanga = at lotus was given at lotus <EOS>
explanation la nala va sonaru katayellama sonaru week end la varen thank you = they gave a at lotus lotus showroom <EOS>
explanation la nala tha kuduthanga details songa  thank you = they gave a good at lotus showroom <EOS>
explanation la nala kudutharu sales man details sonaga free time la vara vanthu vandi yadukaren nane call pandren varu pothu thank you = they gave a at lotus lotus showroom <EOS>
thavayana yellama sonaga details yellama kuduthanga nxt week kula vandi ya yadukaren thank you = they gave a at lotus lotus <EOS>
salesman nallatha explain pannanga eppo vandi eadukara mari illa eadukum pothu angave vanthu eaduthukaran = they lotus lotus was given at lotus showroom <EOS>
sales person Neata attend pannaga vandi Vilakkam nalla Sonnaga Entha Problem Illa Testride Koduthaga Amount Ready pannittu 2 or 3 days la Varen = they gave a at lotus lotus showro

In [ ]:
vocab_tokenization

In [ ]:
final_sent = []
for word in new[0].strip().split(' '):
  word_took = word+'</w>'

  sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
  sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

  #print(sorted_tokens)

  word_given = word_took

  print('Tokenizing word: {}...'.format(word_given))
  if word_given in vocab_tokenization:
      print('Tokenization of the known word:')
      print(vocab_tokenization[word_given])
      print('Tokenization treating the known word as unknown:')
      print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
      #final_sent = final_sent+" "+ str(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
      final_sent.append(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
  else:
      print('Tokenizating of the unknown word:')
      print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
      #final_sent = final_sent+" "+ str(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
      final_sent.append(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

Tokenizing word: quotation</w>...
Tokenization of the known word:
['quotation</w>']
Tokenization treating the known word as unknown:
['quotation</w>']
Tokenizing word: matu</w>...
Tokenizating of the unknown word:
['ma', 'tu</w>']
Tokenizing word: tha</w>...
Tokenization of the known word:
['tha</w>']
Tokenization treating the known word as unknown:
['tha</w>']
Tokenizing word: vangirukom</w>...
Tokenizating of the unknown word:
['van', 'g', 'iru', 'k', 'om</w>']
Tokenizing word: </w>...
Tokenizating of the unknown word:
['</w>']
Tokenizing word: vandi</w>...
Tokenization of the known word:
['vandi</w>']
Tokenization treating the known word as unknown:
['vandi</w>']
Tokenizing word: yadukara</w>...
Tokenizating of the unknown word:
['a', 'd', 'u', 'kara', '</w>']
Tokenizing word: idea</w>...
Tokenizating of the unknown word:
['i', 'de', 'a', '</w>']
Tokenizing word: la</w>...
Tokenization of the known word:
['la</w>']
Tokenization treating the known word as unknown:
['la</w>']
Tokenizi

'quotation ma tu tha van g iru k om  vandi a d u kara  i de a  la i la e p la na t i o n  la nala tha kuduthanga'

In [ ]:
row = "qutoation"
output_words, attentions = evaluate(encoder1, attn_decoder1, row)
output_sentence = ' '.join(output_words)
print(row,'=', output_sentence)

qutoation = they gave a good response at lotus <EOS>


In [ ]:
#for every unknow word, "it learnt they gave a good response". for every empty string, it learnt "they were the vehicle".
for row in new[0]:
    output_words, attentions = evaluate(encoder1, attn_decoder1, row)
    output_sentence = ' '.join(output_words)
    print(row,'=', output_sentence)

q = they gave a good response <EOS>
u = they gave a good response at lotus <EOS>
o = they gave a good response at lotus <EOS>
t = they gave a good response <EOS>
a = they gave a good response at lotus <EOS>
t = they gave a good response at lotus <EOS>
i = they gave a good response <EOS>
o = they gave a good response at lotus <EOS>
n = they gave a good response <EOS>
  = they were the vehicle <EOS>
m = they gave a good response <EOS>
a = they gave a good response <EOS>
t = they gave a good response at lotus <EOS>
u = they gave a good response <EOS>
  = they were the vehicle <EOS>
t = they gave a good response at lotus <EOS>
h = they gave a good response at lotus <EOS>
a = they gave a good response at lotus <EOS>
  = they were the vehicle <EOS>
v = they gave a good response at lotus <EOS>
a = they gave a good response at lotus <EOS>
n = they gave a good response at lotus <EOS>
g = they gave a good response at lotus <EOS>
i = they gave a good response <EOS>
r = they gave a good response <

In [ ]:
final_sent = []
for word in new[0].strip().split(' '):
  word_given = word+'</w>'

  if word_given in vocab_tokenization:
      final_sent.append(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
  else:
      final_sent.append(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
outp = " ".join([" ".join(word) for word in final_sent]).replace("</w>","")

output_words, attentions = evaluate(encoder1, attn_decoder1, outp)
output_sentence = ' '.join(output_words)
print(outp,'=', output_sentence)

quotation ma tu tha van g iru k om  vandi a d u kara  i de a  la i la e p la na t i o n  la nala tha kuduthanga = at lotus was given at lotus <EOS>


In [ ]:
outp

'quotation ma tu tha van g iru k om  vandi a d u kara  i de a  la i la e p la na t i o n  la nala tha kuduthanga'

### for whole remaining 900 rows

In [ ]:
data2  = pd.read_excel('tvs_tamil_Total100_06122020.xlsx', index_col=None)
data2.head()

In [ ]:
verb = data2['Verbatim']

In [ ]:
# from googletrans import Translator
# translator = Translator()

In [ ]:
# translator.detect("varen show room la vandi, ku discount romba kammiya iruku").lang

In [ ]:
# verb = ["next week varen show room la vandi ku discount romba kammiya iruku ;",
#        "next week varen show room la vandi ku discount romba kammiya iruku ",
#        "next week varen show room la vandi, ku discount romba kammiya iruku ",
#         "sales person behavior good. some personal issue so come later",
#        "next week varen show room la vandi ku discount romba kammiya iruku.",
#        "next week varen show room la vandi ku discount. romba kammiya iruku"]

In [ ]:
# if translator.detect(row).lang == 'en':
#         print("english")

In [ ]:
# for row in verb:
#     row = row.replace(",",";")
#     row = row.replace(";;",";").strip()
#     if row[-1] != ".":
#         row = row.replace(".",";")
#     print(row.split(";"))

In [ ]:
# working code
# final = []
# for row in verb:
#     subrows =  row.split(";")
#     out = []
#     for subrow in subrows[:len(subrows)-1]:
#         subrow= re.sub(r'[^A-Za-z]+', ' ', subrow.lower())
#         output_words, attentions = evaluate(encoder1, attn_decoder1, subrow)
#         output_sentence = ' '.join(output_words)
#         out.append(output_sentence.replace("<EOS>",""))
#         output = ";".join(out)
#     final.append(output)

In [ ]:
final = []
for row in verb:
    row = row.replace('.',";")
    row = row.replace(",",";")
    subrows =  row.split(";")
    if len(subrows) != 1:
        subrows = subrows[:len(subrows)-1]
    if len(subrows) == 1 and len(subrows[0].split(" ")) > 39:
        n = len(subrows[0])//2
        subrows = [subrows[0][i:i+n] for i in range(0, len(subrows[0]), n)]
    out = []
#     print(len(subrows))
#     print(subrows)
    for subrow in subrows:
        subrow= re.sub(r'[^A-Za-z]+', ' ', subrow.lower())
        #print(subrow)
        #print(len(subrow.split(" ")),"\n")
        output_words, attentions = evaluate(encoder1, attn_decoder1, subrow)
        output_sentence = ' '.join(output_words)
        out.append(output_sentence.replace("<EOS>",""))
    output = ";".join(out)
    final.append(output)

In [ ]:
len(final)

In [ ]:
final_2000 = pd.DataFrame()

In [ ]:
final_2000['Tamil'] = data2['Verbatim']

In [ ]:
final_2000['Translated'] = final

In [ ]:
final_2000

In [ ]:
final_2000.to_excel("2000_pred_translations_non_lotus.xlsx")

### on Unseen data

In [ ]:
ss = ["strsdfdsfds","ss","rhrhr",'lrkjrt','ertrtre']

In [ ]:
ss[:len(ss)-1]

In [ ]:
ls

In [ ]:
unseen_data = pd.read_excel("tvs_tamil_Total100_06122020.xlsx")

In [ ]:
tamil_test_d = unseen_data['Verbatim'][99:]

In [ ]:
tamil_test_d

In [ ]:
tamil_test_d = [row.strip().split(";") for row in tamil_test_d]
tamil_test_data = [text.strip() for row in tamil_test_d for text in row if text]

In [ ]:
output_words, attentions= evaluate(encoder1, attn_decoder1,"nalla test kuduthanga nalla nalla response")

In [ ]:
output_sentence = ' '.join(output_words)
print('<', output_sentence)

In [ ]:
output_words, attentions = evaluate(encoder1, attn_decoder1, "nalla response panni neata attend pannaga vandi vilakkam nalla sonnaga problem diwali ku varen")
output_sentence = ' '.join(output_words)
print('<', output_sentence)

In [ ]:
output_words, attentions = evaluate(encoder1, attn_decoder1, "nalla response attend vandi nalla diwali ku varen")
output_sentence = ' '.join(output_words)
print('<', output_sentence)

In [ ]:
output_words, attentions = evaluate(encoder1, attn_decoder1, "nalla")
output_sentence = ' '.join(output_words)
print('<', output_sentence)

### Its throwing key error for the words whcich the model dont have its embedding in vector matrix. So if we skip the word and presents it as it is, and skip to next word, then we are missing the sequence. so we are getting bad results as its sequence is important.

#### Solution from the blog [here]("https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html") is using pretrained Word2vec or glove. Finetunning steps in the same blog as well.

##### Above Keyerror got solved with the solution from  [here](https://github.com/qlanners/nmt_tutorial/blob/master/nmt_tutorial.ipynb)

In [ ]:
pred = []
for ind,row in enumerate(data["Tamil_data"]):
    output_words, attentions = evaluate(encoder1, attn_decoder1, row)
    pred.append(" ".join(output_words))

In [ ]:
data['Predicted_data'] = pred

In [ ]:
data["Predicted_data"]

In [ ]:
data.to_excel("pred_results_100_seq2seq.xlsx")

## FINAL OUTPUT

In [ ]:
data

### FINAL OUTPUT on UNSEEN DATA

In [ ]:
unseen_data[['Area', 'rating', 'L1_FirstDriver', 'L2_FirstDriver', 'L1_SecondDriver','L2_SecondDriver', 'L1_ThirdDriver', 'L2_ThirdDriver', 'Verbatim','ENGLISH TRANSLATION']].head()

In [ ]:
ver = []
for ind in range(99,len(unseen_data[['Verbatim']])+1):
    if ind == 2012:
        break
    text = unseen_data[['Verbatim']].iloc[ind].values[0]
    output = re.sub(r'[^A-Za-z]+', ' ', text.lower())
    ver.append(output)

In [ ]:
unseen_data['Verbatim'][99:] = ver

In [ ]:
unseen_data.tail()

In [ ]:
pred1 = []
for ind,row in enumerate(unseen_data["Verbatim"]):
    print(row)
    output_words, attentions = evaluate(encoder1, attn_decoder1, row)
    pred1.append(" ".join(output_words))

### How to handle out of vocabulary issue?
- https://www.coursera.org/lecture/language-processing/how-to-deal-with-a-vocabulary-mvV6t
- https://github.com/shabeelkandi/Handling-Out-of-Vocabulary-Words-in-Natural-Language-Processing-using-Language-Modelling
- https://github.com/mjc92/CopyNet
- https://medium.com/@shabeelkandi/handling-out-of-vocabulary-words-in-natural-language-processing-based-on-context-4bbba16214d5


In [ ]:
output_words, attentions = evaluate(encoder1, attn_decoder1, "inance konjam problem ahh ulladhu")
output_sentence = ' '.join(output_words)
print('<', output_sentence)

In [ ]:
output_words, attentions = evaluate(encoder1, attn_decoder1, "showroom nalla iruku ")
output_sentence = ' '.join(output_words)
print('<', output_sentence)